# Implementación de un algoritmo evolutivo para resolver el problema del sudoku


El primer paso consiste en leer la matriz de 9x9 de un archivo .csv

In [ ]:
import pandas
sudoku = pandas.read_csv('sudoku.csv',header=None).values
print(sudoku)

El siguiente paso consiste en calcular los números faltantes en cada renglón

In [ ]:
import numpy as np
nums = np.arange(10) # Arreglo con los números del 0 al 9
print(nums)
print(sudoku[0])

In [ ]:
nums = set(nums)     # Convertir a set
sr = set(sudoku[0])  # Convertir a set
missing_nums= nums.difference(sr) # Diferencia de conjuntos
print(missing_nums)

In [ ]:
# Calcular los números faltantes en cada uno de los renglones
nums = set(np.arange(10))
missing_nums = []
for i in range(9):
    sr = set(sudoku[i])
    missing_nums.append( list(nums.difference(sr)) )
print(missing_nums)

### Creación aleatoria de invidiuos 
Cada individuo se genera como una permutación aleatoria de los números faltantes

In [ ]:
ind = []
for mn in missing_nums:
    mnind = np.copy(mn)
    np.random.shuffle(mnind) 
    ind.append(mnind)
print(ind)

In [ ]:
def create_new_individual(missing_nums):
    ind = []
    for mn in missing_nums:
        mnind = np.copy(mn)
        np.random.shuffle(mnind)
        ind.append(mnind)
    return ind
ind = create_new_individual(missing_nums)
print(ind)

In [ ]:
# Crear una copia de la matriz sudoku para crear una matriz del individuo
mind = np.copy(sudoku)
print(mind[0,:])
print(mind[0,:]==0) # Las casillas que son 0 en el primer renglón

In [ ]:
mind = np.copy(sudoku)
idx = mind[0,:]==0
mind[0,idx] = ind[0] # Remplazar los 0's con los valores del individuo
print('ind')
print(ind[0])
print('mind')
print(mind)

In [ ]:
# Calcular la matriz del sudoku poniendo los valores del individuo
def calculate_individual_matrix(ind):
    mind = np.copy(sudoku)
    for i in range(9):
        idx = sudoku[i, :]==0
        mind[i, idx] = ind[i]
    return mind
ind = create_new_individual(missing_nums)
mind = calculate_individual_matrix(ind)
print('sudoku')
print(sudoku)
print('ind')
print(ind)
print('mind')
print(mind)

In [ ]:
def print_individual(ind):
    mind = calculate_individual_matrix(ind)
    s = ''
    for i in range(9):
        for j in range(9):
            s+= '|'+str(mind[i,j]) if j%3==0 else ' '+str(mind[i,j])
        s += '\n'
        if (i+1)%3==0 and i<8:
            s+= '------------------\n'
    print(s)
ind = create_new_individual(missing_nums)
print_individual(ind)

### Cálculo de la aptitud (fitness)
La aptitud de un individuo se calcula a partir del número de errores (números duplicados) en los cuadrantes y en las columnas. Las filas no se toman en cuenta porque por definición los individuos son representados como permutaciones de los números faltantes en las filas.

In [ ]:
# Imprimir la lista de los números únicos del primer renglón
ind = create_new_individual(missing_nums)
mind = calculate_individual_matrix(ind)
print(mind)
print(np.unique(mind[:,0]))

In [ ]:
# Calcular el número de valores que faltan, igual al número de valores que se repiten
9-len(np.unique(mind[:,0]))

In [ ]:
# Errores en las columnas
fit = 0
for c in range(9):
    fit += 9-len(np.unique(mind[:,c]))
print(fit)

In [ ]:
# Obtener un cuadrante de la matriz
print( mind )
r = 0
c = 0
print( mind [r*3:r*3+3,c*3:c*3+3] )

In [ ]:
# Errores en los cuadrantes
fit = 0
for r in range(3):
    for c in range(3):
        nums = np.unique( np.reshape(mind[r*3:r*3+3,c*3:c*3+3],(9)) )
        fit += 9 - len(nums)
print(fit)

In [ ]:
def calculate_individual_fitness(ind):
    fit = 0
    mind = calculate_individual_matrix(ind)
    for c in range(9):
        fit += 9-len(np.unique(mind[:,c]))
    for r in range(3):
        for c in range(3):
            nums = np.unique( np.reshape(mind[r*3:r*3+3,c*3:c*3+3],(9)) )
            fit += 9 - len(nums)
    return fit
fit = calculate_individual_fitness(ind)
print(fit)

### Reproducción
La reproducción tiene el objetivo de mezclar las características de dos padres. 
En este caso utilizaremos la metodología punto de cruza descrito en el aula virtual

In [ ]:
p1 = create_new_individual(missing_nums)
p2 = create_new_individual(missing_nums)
print(p1)
print(p2)

In [ ]:
# Punto de cruza aleatorio
c = np.random.randint(1, 8)
print(c)

In [ ]:
def crossover(p1,p2):
    o = []
    c = np.random.randint(1, 8)
    for i in range(0,c):
        o.append( np.copy(p1[i]) )
    for i in range(c,9):
        o.append( np.copy(p2[i]) )
    return o
o = crossover(p1,p2)
print(o)

### Mutación
La mutación tiene el objetivo de variar un poco un individuo. En este caso lo que haremos es elegir al azar uno de los renglones del individuo, después, también de forma aleatoria, seleccionar dos números faltantes, y lo que se realizará es intercambiar sus posiciones

In [ ]:
ind = create_new_individual(missing_nums)
r = np.random.randint(9)
n1 = np.random.randint(len(ind[r]))
n2 = np.random.randint(len(ind[r]))
print('ind',ind)
print('r:',r,'n1:',ind[r][n1],'n2:',ind[r][n2])

In [ ]:
def mutation(ind):
    r = np.random.randint(9)
    n1 = np.random.randint(len(ind[r]))
    n2 = np.random.randint(len(ind[r]))
    ind[r][n1], ind[r][n2] = ind[r][n2], ind[r][n1]
    print('r:',r,'n1:',ind[r][n1],'n2:',ind[r][n2])
    return ind
print(ind)
ind = mutation(ind)
print(ind)

### Evolución
Finalmente, hay que unir todas las piezas de código 

In [ ]:
# Parámetros
N = 100    # tamaño de la población
Pc = 0.7   # probabilidad de cruza
Pm = 0.3   # probabilidad de mutación
G = 1000   # número de generaciones

# Leer el archivo
sudoku = pandas.read_csv(file, header=None).values

# Proceso evolutivo
population = init_population(sudoku,N)
fitness = calculate_fitness(population)
ind_elite, fit_elite = elite(population,fitness)

g = 0
print('Generation:',g,' fitness:',fit_elite)
while fit_elite>0 and g<G:
    population = selection(population,fitness)
    population = crossover(population,Pc)
    population = mutation(population,Pm)
    fitness = calculate_fitness(population)
    population, fitness, ind_elite, fit_elite = elite(population,fitness,ind_elite,fit_elite)
    g+=1
    if g%10==0:
        print('Generation:',g,' fitness:',fit_elite)
print('Generation:',g,' fitness:',fit_elite)
print_individual(ind_elite)

init_population: Crea una población de N individuos aleatorios, population es una lista de individuos.

calculate_fitness: Calcula la aptitud (fitness) de todos los individuos en la población y regresa un arreglo con el fitness de todos los individuos

elite: Encuentra y guarda una copia del mejor individuo de la población. 
Si aún no hay ningún individuo elite (primera vez que se manda llamar), sólo regresa el individuo elite y su aptitud.
Si ya hay un individuo elite hace lo siguiente. Si el mejor individuo en la población actual es mejor que el individuo elite, entonces el individuo elite y su fitness son actualizados. En otro caso, el individuo elite reemplaza a un individuo, seleccionado de forma aleatoria, de la población. 

selection: Recibe como parámetro la población y el arreglo de aptitudes; y regresa una nueva población de individuos aptos. Cada individuo en la nueva población es seleccionado utilizando torneo binario, es decir, dos individuos de la población anterior son seleccionados de forma aleatoria y el mejor de los dos es seleccionado para formar parte de la nueva población. El tamaño de la población nueva es del mismo tamaño que la población anterior.

crossover: Recibe como parámetro la población de individuos seleccionados y la probabilidad de cruza; y regresa una nueva población de individuos hijos. Los individuos de la nueva población son creados uno por uno como sigue. Primero, se genera un número aleatorio del 0 al 1. Si el número es menor o igual a la probabilidad de cruza, entonces el individuo nuevo es producto de la cruza de dos individuos seleccionados al azar de la población de individuos seleccionados. Si el número no es menor o igual a Pc, entonces el nuevo individuo es una copia de un individuo seleccionado al azar. 

mutation: Recibe como parámetro la población de individuos y la probabilidad de mutación; y regresa una nueva población de individuos. Los individuos de la nueva población son mutados uno por uno como sigue. Primero, se genera un número aleatorio del 0 al 1. Si el número es menor o igual a la probabilidad de mutación, entonces el individuo es mutado, si no, el individuo no se modifica.